In [1]:
install.packages("disk.frame", repo="https://cran.rstudio.com/")

library(tidyverse)
library(caret)
library(nnet)
library(reticulate)
sagemaker <- import('sagemaker')
session <- sagemaker$Session()
bucket <- session$default_bucket()

also installing the dependencies ‘pryr’, ‘fst’

Warning message in install.packages("disk.frame", repo = "https://cran.rstudio.com/"):
“installation of package ‘pryr’ had non-zero exit status”Warning message in install.packages("disk.frame", repo = "https://cran.rstudio.com/"):
“installation of package ‘fst’ had non-zero exit status”Warning message in install.packages("disk.frame", repo = "https://cran.rstudio.com/"):
“installation of package ‘disk.frame’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3

In [2]:
# Load the data
triagedata <- get(load(file = "ScaledNumericData.RData"))
#head(data)
#data <- as.dataframe(data)
#sapply(data, class)
str(triagedata)
summary(triagedata)
dim(triagedata)

'data.frame':	556298 obs. of  76 variables:
 $ ID                          : chr  "d5db547ae8e87d127ce0d4e51447befa5ff5012b_2019-08-04T02:50:00Z" "8ba1ffcd025f4002f0a94b9e5414a8868b40604f_2018-01-05T00:52:00Z" "75ccf97f6dbc795bd0bfc34067ab6b1a3d7e9325_2017-05-01T05:00:00Z" "ce3585171e5bf2dbaa366ebb9157cdbd790bcd66_2017-02-10T00:34:00Z" ...
 $ TRIAGE_CATEGORY             : num  3 3 2 3 3 3 3 2 4 3 ...
 $ AGE                         : num  0.331 0.29 0.537 0.824 0.331 ...
 $ ADMISSION_WITHIN_48_HOURS   : num  -0.26 -0.26 -0.26 -0.26 -0.26 ...
 $ AVPU_1                      : num  -0.1489 0.0287 0.0111 -0.1489 -0.1489 ...
 $ AVPU_1_DT_TM                : num  0.0307 -0.02648 -0.02648 -0.00368 0.01594 ...
 $ GCS_1                       : num  0.060655 0.000335 0.003378 0.256747 0.003378 ...
 $ GCS_1_DT_TM                 : num  -0.06299 -0.06299 -0.06299 -0.00181 -0.06299 ...
 $ RR_1                        : num  -0.4047 -0.2827 0.5204 -0.0963 -0.4047 ...
 $ RR_1_DT_TM                  : n

      ID            TRIAGE_CATEGORY      AGE          ADMISSION_WITHIN_48_HOURS
 Length:556298      Min.   :1.000   Min.   :-1.7236   Min.   :-0.2595          
 Class :character   1st Qu.:2.000   1st Qu.:-0.7373   1st Qu.:-0.2595          
 Mode  :character   Median :3.000   Median :-0.1208   Median :-0.2595          
                    Mean   :3.156   Mean   : 0.0000   Mean   : 0.0000          
                    3rd Qu.:4.000   3rd Qu.: 0.7832   3rd Qu.:-0.2595          
                    Max.   :5.000   Max.   : 2.6736   Max.   : 3.8528          
     AVPU_1           AVPU_1_DT_TM           GCS_1           
 Min.   :-0.148948   Min.   :-31.69604   Min.   :-22.484970  
 1st Qu.:-0.148948   1st Qu.: -0.02648   1st Qu.:  0.003378  
 Median :-0.148948   Median : -0.02461   Median :  0.143044  
 Mean   : 0.000000   Mean   :  0.00000   Mean   :  0.000000  
 3rd Qu.: 0.000915   3rd Qu.: -0.01004   3rd Qu.:  0.256747  
 Max.   :17.635976   Max.   :235.40949   Max.   :  0.256747  
  GCS_

[1] 556298     76

In [3]:
library(dplyr)
#ED_data = select(data,-ID)
triagedata$ID <- NULL
ED_data = triagedata %>% 
  #select(-ends_with('_2')) %>%
  select(-ends_with('1_DT_TM')) %>%
  select(-ends_with('2_DT_TM')) %>%
  select(TRIAGE_CATEGORY, everything()) 
ED_data$TRIAGE_CATEGORY = as.numeric(ED_data$TRIAGE_CATEGORY)
str(ED_data)

'data.frame':	556298 obs. of  63 variables:
 $ TRIAGE_CATEGORY             : num  3 3 2 3 3 3 3 2 4 3 ...
 $ AGE                         : num  0.331 0.29 0.537 0.824 0.331 ...
 $ ADMISSION_WITHIN_48_HOURS   : num  -0.26 -0.26 -0.26 -0.26 -0.26 ...
 $ AVPU_1                      : num  -0.1489 0.0287 0.0111 -0.1489 -0.1489 ...
 $ GCS_1                       : num  0.060655 0.000335 0.003378 0.256747 0.003378 ...
 $ RR_1                        : num  -0.4047 -0.2827 0.5204 -0.0963 -0.4047 ...
 $ O2SATS_1                    : num  0.9333 -0.0826 -3.9544 -2.8682 0.3902 ...
 $ FIO2_1                      : num  -0.0553 -0.0448 -0.0497 0.9193 -0.0497 ...
 $ PULSE_1                     : num  0.258 -0.227 0.763 -0.292 -0.751 ...
 $ HEART_RATE_1                : num  -0.37 -0.377 -0.433 -0.415 -0.433 ...
 $ SBP_1                       : num  -0.145 0.242 -0.241 -0.145 1.298 ...
 $ DBP_1                       : num  0.604 0.567 0.604 -0.288 -0.824 ...
 $ MAP_1                       : num  0.15

In [5]:
names(ED_data)[names(ED_data) == "ENT, face"] <- "ENT_face"
names(ED_data)[names(ED_data) == "Mental Health"] <- "Mental_Health"
names(ED_data)[names(ED_data) == "Obstetrics and Gynaecology"] <- "Obstetrics_Gynaecology"

In [6]:
# Split the data into training and test set
set.seed(123)
training.samples <- ED_data$TRIAGE_CATEGORY %>% 
  createDataPartition(p = 0.7, list = FALSE)
train.data  <- ED_data[training.samples, ]
test.data <- ED_data[-training.samples, ]

In [7]:
# Fit the model
model <- nnet::multinom(TRIAGE_CATEGORY ~., data = train.data)
# Summarize the model
summary(model)
# Make predictions
predicted.classes <- model %>% predict(test.data)
head(predicted.classes)
# Model accuracy
mean(predicted.classes == test.data$TRIAGE_CATEGORY)

# weights:  320 (252 variable)
initial  value 626731.217480 
iter  10 value 435901.694893
iter  20 value 425682.226195
iter  30 value 423999.349403
iter  40 value 423495.128836
iter  50 value 423199.444687
iter  60 value 422771.270520
iter  70 value 422315.579456
iter  80 value 421817.583371
iter  90 value 421109.321460
iter 100 value 420405.816490
final  value 420405.816490 
stopped after 100 iterations


Call:
nnet::multinom(formula = TRIAGE_CATEGORY ~ ., data = train.data)

Coefficients:
  (Intercept)         AGE ADMISSION_WITHIN_48_HOURS      AVPU_1     GCS_1
2    2.174647  0.18807688              -0.055629905 -0.04384273 0.2857296
3    2.593055 -0.05472528              -0.039406069 -0.12390695 0.4222277
4    2.213508 -0.35775937              -0.003651422 -0.13050256 0.4132581
5   -1.105538 -0.51551458               0.093965813 -0.10192620 0.3756070
         RR_1     O2SATS_1      FIO2_1     PULSE_1 HEART_RATE_1       SBP_1
2 -0.06367685 -0.015847137 -0.10738193  0.13730402  -0.11024146  0.31215668
3 -0.40070363 -0.007117044 -0.09912458 -0.01223149  -0.10335577  0.17221163
4 -0.77019641 -0.063500355 -0.30165055 -0.20834596  -0.05493849  0.03532019
5 -0.55827183 -0.092979374 -0.25992329 -0.43001492   0.04154172 -0.02525074
      DBP_1      MAP_1 TEMP_TYMP_1 TEMP_ORAL_1 PATIENT_HEIGHT PATIENT_WEIGHT
2 0.1383327 -0.3490036  0.11245491  0.03432428      -1.032230      0.8537966
3 0.143808

[1] 3 3 3 3 3 3
Levels: 1 2 3 4 5

[1] 0.5288037

In [8]:
mean(predicted.classes == test.data$TRIAGE_CATEGORY)

[1] 0.5288037

In [10]:
# Predicting the values for train dataset
train.data$precticed <- predict(model, newdata = train.data, "class")

# Building classification table
ctable <- table(train.data$TRIAGE_CATEGORY, train.data$precticed)

# Calculating accuracy - sum of diagonal elements divided by total obs
round((sum(diag(ctable))/sum(ctable))*100,2)

[1] 53.06